In [1]:
import os
import joblib
import unet
from glob import glob
import torch
import warnings
from tqdm import tqdm
import numpy as np
from skimage.io import imread, imsave
import matplotlib.pyplot as plt

In [2]:
args = joblib.load('models/jiu0Monkey_Unet_woDS/args.pkl')

joblib.dump(args, 'models/jiu0Monkey_Unet_woDS/args.pkl')
model = unet.__dict__[args.arch](args)
model = model.cuda()


In [3]:
data1 = np.load('/data0/BraTS2020/testImage/BraTS20_Validation_113_88.npy')
data1 = data1.T
print(data1.shape)
data1 = np.expand_dims(data1,0)
#data1.reshape()
print(data1.shape)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
img_ = torch.from_numpy(data1).type(torch.FloatTensor) 
img = img_.to(device=device, dtype=torch.float32)
model.load_state_dict(torch.load('models/jiu0Monkey_Unet_woDS/model.pth'))
model.eval()

(4, 160, 160)
(1, 4, 160, 160)


Unet(
  (down1): Downsample_block(
    (conv1): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (down2): Downsample_block(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (down3): Downsample_block(
    (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(256, 256, kernel_size=

In [4]:
# val_dataset = Dataset(args,val_img_paths)
# val_loader = torch.utils.data.DataLoader(
#         val_dataset,
#         batch_size=args.batch_size,
#         shuffle=False,
#         pin_memory=True,
#         drop_last=False)

In [5]:
 with torch.no_grad():
    img = img.cuda()
    output = model(img)
    #print("img_paths[i]:%s" % img_paths[i])
    output = torch.sigmoid(output).data.cpu().numpy()
    print(output.shape)
    np.save('test.npy',output)

(1, 3, 160, 160)


In [6]:
# with warnings.catch_warnings():
#             warnings.simplefilter('ignore')

#             with torch.no_grad():
#                 for i, (input) in tqdm(enumerate(val_loader), total=len(val_loader)):
#                     input = input.cuda()
#                     #target = target.cuda()

#                     # compute output
#                     if args.deepsupervision:
#                         output = model(input)[-1]
#                     else:
#                         output = model(input)
#                     #print("img_paths[i]:%s" % img_paths[i])
#                     output = torch.sigmoid(output).data.cpu().numpy()
#                     img_paths = val_img_paths[args.batch_size*i:args.batch_size*(i+1)]

In [7]:
for i in range(output.shape[0]):
    """
    生成灰色圖片
    wtName = os.path.basename(img_paths[i])
    overNum = wtName.find(".npy")
    wtName = wtName[0:overNum]
    wtName = wtName + "_WT" + ".png"
    imsave('output/%s/'%args.name + wtName, (output[i,0,:,:]*255).astype('uint8'))
    tcName = os.path.basename(img_paths[i])
    overNum = tcName.find(".npy")
    tcName = tcName[0:overNum]
    tcName = tcName + "_TC" + ".png"
    imsave('output/%s/'%args.name + tcName, (output[i,1,:,:]*255).astype('uint8'))
    etName = os.path.basename(img_paths[i])
    overNum = etName.find(".npy")
    etName = etName[0:overNum]
    etName = etName + "_ET" + ".png"
    imsave('output/%s/'%args.name + etName, (output[i,2,:,:]*255).astype('uint8'))
    """
    rgbPic = np.zeros([160, 160, 3], dtype=np.uint8)
    for idx in range(output.shape[2]):
        for idy in range(output.shape[3]):
            if output[i,0,idx,idy] > 0.5:
                rgbPic[idx, idy, 0] = 0
                rgbPic[idx, idy, 1] = 128
                rgbPic[idx, idy, 2] = 0
            if output[i,1,idx,idy] > 0.5:
                rgbPic[idx, idy, 0] = 255
                rgbPic[idx, idy, 1] = 0
                rgbPic[idx, idy, 2] = 0
            if output[i,2,idx,idy] > 0.5:
                rgbPic[idx, idy, 0] = 255
                rgbPic[idx, idy, 1] = 255
                rgbPic[idx, idy, 2] = 0
    rgbPic = np.array(rgbPic)
    print(rgbPic.shape)
    for i in range(3):
        rgbPic[:,:,i]=rgbPic[:,:,i].T
    imsave('test1.png',rgbPic)

(160, 160, 3)
